# EEG Quality Index

This notebook takes you through the implementation 

## Import libraries

In [1]:
%matplotlib qt

## Import libraries
import numpy as np
import moabb
import scipy.fft as fft
import scipy.stats as stats

## Import and visualize data

In [2]:
# Settings
plot_raw = False    # Boolean to plot raw data

# Set log level
moabb.set_log_level("warning")

# Import dataset
from moabb.datasets import BNCI2014001

dataset = BNCI2014001()

sessions = dataset.get_data(subjects=[1])
subject = 1
session_name = "session_T"
run_name = "run_1"
raw = sessions[subject][session_name][run_name]


# Visualize RAW data
if plot_raw:
    raw.plot()

## Signal quality metrics

The signal is analyzed as a sliding window of 1 second, with steps of 1 sample. The windows are defined as $\pm$ 500 msec before and after the center of the window. This window size allows to keep a 1 Hz frequency resolution.

First, the data will be converted to a `numpy` array with the previously mentioned sliding window approach.

In [3]:
# Separate data 
eeg_raw = raw.get_data()
srate = raw.info['sfreq']

window = int(srate) # Number of samples for each window [n]
slide = 1           # Number of samples to slide each window [n]

def sliding_window(data, window, slide):
    """
        This function calculates a sliding window across the longest dimension of a 2D array and returns a 
        3D tensor where the 3rd dimension are the windows

    Parameters
    ----------
        data: 2D array_like
            Data to be divided into sliding windows
        window: int
            Number of samples for each window
        slide: int
            Number of samples to slide each window
    Returns
    -------
        data_windowed: 3D array_like   
    """
    # If column vectors, transpose for row vectors
    shape = data.shape
    if shape[0] > shape[1]:
        transposed_data = True 
        data = data.T
        shape = data.shape

    n_chans = shape[0]                      # Number of channels [n]
    data_length = shape[1]                  # Input data length [n]                 
    max_n_windows = data_length-window+1    # Max number of windows required [n]

    # Create matrix for indices of the sliding window
    window_idy = np.arange(0,max_n_windows,slide).reshape(-1,1).repeat(window, axis=1)  # Index for columns
    window_idx = np.arange(window).reshape(1,-1).repeat(np.size(window_idy,0), axis=0)  # Index for rows
    window_mat = window_idx + window_idy                                                # Matrix with index of windows
    window_shape = window_mat.shape

    # Preallocate output data
    data_windowed = np.zeros((n_chans, window_shape[1], window_shape[0]))

    # Fill each window with the right index values
    for w in range(window_shape[0]):
        data_windowed[:,:,w] = data[:,window_mat[w,:].astype(int)]
    
    return data_windowed

eeg_windowed = sliding_window(eeg_raw, window, slide)

### Averaged single-sided amplitude spectrum (1-50 Hz) 

The frequency range of 1-50Hz covers standard brain-wave bands  of  delta,  (1-4Hz),  theta  (4-8Hz),  alpha  (8-12Hz),beta (12-30Hz),  and  low  gamma  (30-50Hz)  waves.  The  average signal-sided   amplitude   spectrum   of   these   bands   can   be determined  from  the  Fast  Fourier  Transform  (FFT)  of  the signal.



In [4]:
# Calculate single sided FFT and frequency vector
single_fft = np.abs(fft.rfft(eeg_windowed, n=window, axis=1, workers=-1))
size_fft = np.shape(single_fft)
freq_vect = srate * np.linspace(0, window/2, size_fft[1]) / window

# Average from f_start to f_end
f_start = 1     # Frequency start for average [Hz]
f_end = 50      # Frequency end for average [Hz]
f_mask = np.expand_dims((freq_vect>=f_start) & (freq_vect<=f_end), axis=0)
f_mask_tensor = np.expand_dims(f_mask.repeat(size_fft[0], axis=0), axis=2).repeat(size_fft[2], axis=2)

mean_ssas = np.mean(single_fft, axis=1, where=f_mask_tensor)    # Mean Single Sided Amplitude Spectrum

### Line noise - average single-sided amplitude spectrum (59-61 Hz range)

Line noise artifact arises from electromagnetic interference with the power grid (60 Hz in North America, 50 Hz in Europe) Most of the induced electrical noise in EEG experiments arises from AC devices near the recording hardware. To measure the strength of the line noise in  the EEG signal, we compute the average single-sided amplitude spectrum of the signal over the range of 59-61 Hz.

In [5]:
# Average from f_start to f_end
f_start = 59    # Frequency start for average [Hz]
f_end = 61      # Frequency end for average [Hz]
f_mask = np.expand_dims((freq_vect>=f_start) & (freq_vect<=f_end), axis=0)
f_mask_tensor = np.expand_dims(f_mask.repeat(size_fft[0], axis=0), axis=2).repeat(size_fft[2], axis=2)

line_ssas = np.mean(single_fft, axis=1, where=f_mask_tensor)    # Mean Single Sided Amplitude Spectrum

### RMS Amplitude

The root-mean-square (RMS) amplitude of the EEG signal is a general measure of the magnitude of the signal throughout the window. RMS of a signal `X` of length `N` can be calculated as

$$ X_{rms} = \sqrt{\frac{1}{N} \sum^{N}_{n=1}{|X_n|^2}} $$

In [6]:
# RMS of each window
window_rms = np.sqrt(np.mean(eeg_windowed**2, axis=1)) 

### Maximum gradient

The maximum gradient of the EEG signal is the largets difference between all adjacent samples within the window. It is defined as:

$$ \text{MG} = \text{max}[x(n) - x(n-1)] $$

This is a commonly used artifact-detection method in EEG analyses, where a step of 10 $\mu \text{V/ms}$ is often used a s threshold to indicate any high amplitude or high frequency artifact that produces a large spike in voltage. 

In [7]:
max_grad = (np.diff(eeg_windowed, axis=1)).max(axis=1) 

### Zero-crossing rate

The zero-crossing rate (ZCR) is the rate at whith the signal changes signs from positive to negative. It is defined as:

$$ \text{ZCR} = \frac{1}{N} \sum^{N}_{n=1} {|\text{sign}[x(n)] - \text{sign}[x(n-1)]|} $$

It is an indicator of the frequency at whit the majority of energy is conocentrades in the signal spectrum. The ZCR should increase in high-frequency artifacts and decrease with low-frequency artifacts that cause the EEG trace to drift away from the zero line.

In [8]:
def zcr(data):
    """
        Compute the zero-crossing rate (ZCR) of the input data along the columns

    Parameters
    ----------
        data: array_like
            Data to compute the ZCR

    Returns
    -------
        data_zcr: array_like
            ZCR result

    Notes
    -----
        - ZCR will be one less dimension than input data
    """

    data_shape = data.shape

    # Make sure data is in a row matrix
    if data_shape[0] > data_shape[1]:
        data = data.T

    data_zcr = np.mean(np.diff(np.sign(eeg_windowed), axis=1), axis=1)

    return data_zcr

eeg_zcr = zcr(eeg_windowed)

# eeg_zcr = np.mean(np.diff(np.sign(eeg_windowed), axis=1), axis=1)

### Kurtosis

Kurtosis is a standard statistical measure of haviness of the tails of a distribution of amples (i.e., it indicates how likely the sample is to contain an outlier). For a window  `X` of length `N`, the kurtosis can be calculated by:

$$ \text{Kurtosis} = \frac{\frac{1}{N} \sum^{N}_{i=1}{(x(i)-\bar{x})^4}} {(\frac{1}{N} \sum^{N}_{i=1}{(x(i)-\bar{x})^2})^2} $$

The higher the likelihood of outlier in the sample, the larget the kutrosis value. Similarly, the more uniform the distribution, the lower the kurtosis relative to a normal distribution. Kurtosis has also been used as a means of artifact detection in EEG signals.

In [9]:
eeg_kurtosis = stats.kurtosis(eeg_windowed, axis=1)

## Scoring

The EEG Quality Index (EQI) for each window is compared to a normative database of artifact-free "clean" EEG. The data is Z-scored with the following values:
- EQI $\le \pm$ 1 stdev = 0.
- $\pm$ 1 stdev $\lt$ EQI $\le \pm$ 2 stdev = 1
- $\pm$ 2 stdev $\lt$ EQI $\le \pm$ 3 stdev = 2
- $\pm$ 3 stdev $\lt$ EQI = 3 

Thus, a score of 0 would represent a segment of EEG that is considered normal, with an increasing score suggesting an increased likelihood of abnormal EEG (i.e., some type of artifact).

With the sliding window approach, the average score for a longer segment of EEG is a funcion of the number and severity of artifacts and the total duration of the signal. A single large artifact in a very short recording might affect the overall quality more than a single large articat over a longer recording. Similarly, a small artifact that occurs over a long period would affect the EQI substantially.